In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arc-prize-2025/arc-agi_training_solutions.json
/kaggle/input/arc-prize-2025/arc-agi_evaluation_solutions.json
/kaggle/input/arc-prize-2025/arc-agi_evaluation_challenges.json
/kaggle/input/arc-prize-2025/sample_submission.json
/kaggle/input/arc-prize-2025/arc-agi_training_challenges.json
/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json


In [2]:
"""
ARC Prize 2025 - Multi-Strategy Solver
Kaggle Submission Notebook By Emmanuel Olimi Kasigazi

Strategy inspired by multi-approach mathematical reasoning:
- Multiple solving strategies attempt each task
- Verification checks consistency with training examples
- Best candidates selected for submission
"""

import json
import numpy as np
from typing import List, Dict, Any, Optional

# ============================================================
# LOAD TEST DATA
# ============================================================

print("Loading test challenges...")
with open('/kaggle/input/arc-prize-2025/arc-agi_test_challenges.json', 'r') as f:
    test_challenges = json.load(f)

print(f"Loaded {len(test_challenges)} test tasks")

# ============================================================
# STRATEGY 1: COLOR MAPPING
# ============================================================

def try_color_mapping(train_examples: List[Dict], test_input: List[List[int]]) -> Optional[List[List[int]]]:
    """
    Detect if transformation is a simple color substitution.
    Maps each input color to an output color consistently.
    """
    try:
        if len(train_examples) == 0:
            return None
        
        # Check first example
        ex = train_examples[0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        # Only works for same-size transformations
        if inp.shape != out.shape:
            return None
        
        # Build color mapping dictionary
        color_map = {}
        for i in range(inp.shape[0]):
            for j in range(inp.shape[1]):
                in_color = int(inp[i, j])
                out_color = int(out[i, j])
                
                if in_color in color_map:
                    if color_map[in_color] != out_color:
                        return None  # Inconsistent mapping
                else:
                    color_map[in_color] = out_color
        
        # Verify mapping works on all training examples
        for ex in train_examples[1:]:
            inp2 = np.array(ex['input'])
            out2 = np.array(ex['output'])
            
            if inp2.shape != out2.shape:
                return None
            
            for i in range(inp2.shape[0]):
                for j in range(inp2.shape[1]):
                    expected = color_map.get(int(inp2[i, j]))
                    actual = int(out2[i, j])
                    if expected != actual:
                        return None
        
        # Apply to test input
        test_arr = np.array(test_input)
        result = np.zeros_like(test_arr)
        for i in range(test_arr.shape[0]):
            for j in range(test_arr.shape[1]):
                result[i, j] = color_map.get(int(test_arr[i, j]), test_arr[i, j])
        
        return result.tolist()
        
    except Exception as e:
        return None

# ============================================================
# STRATEGY 2: GRID TILING/REPETITION
# ============================================================

def try_tiling(train_examples: List[Dict], test_input: List[List[int]]) -> Optional[List[List[int]]]:
    """
    Detect if output is input repeated/tiled in a pattern.
    Example: 2x2 input becomes 6x6 output (tiled 3x3)
    """
    try:
        if len(train_examples) == 0:
            return None
        
        ex = train_examples[0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        # Calculate tiling ratio
        if inp.shape[0] == 0 or inp.shape[1] == 0:
            return None
        
        h_ratio = out.shape[0] // inp.shape[0]
        w_ratio = out.shape[1] // inp.shape[1]
        
        if h_ratio < 1 or w_ratio < 1:
            return None
        
        # Check if exact tiling
        expected = np.tile(inp, (h_ratio, w_ratio))
        if expected.shape != out.shape or not np.array_equal(expected, out):
            return None
        
        # Verify on all training examples
        for ex in train_examples[1:]:
            inp2 = np.array(ex['input'])
            out2 = np.array(ex['output'])
            expected2 = np.tile(inp2, (h_ratio, w_ratio))
            
            if expected2.shape != out2.shape or not np.array_equal(expected2, out2):
                return None
        
        # Apply to test
        test_arr = np.array(test_input)
        result = np.tile(test_arr, (h_ratio, w_ratio))
        return result.tolist()
        
    except Exception as e:
        return None

# ============================================================
# STRATEGY 3: SPATIAL TRANSFORMATIONS
# ============================================================

def try_spatial_transform(train_examples: List[Dict], test_input: List[List[int]]) -> Optional[List[List[int]]]:
    """
    Detect rotations, flips, and transpose operations.
    """
    try:
        if len(train_examples) == 0:
            return None
        
        ex = train_examples[0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        # Only for same-size transformations
        if inp.shape != out.shape:
            return None
        
        # Try 90, 180, 270 degree rotations
        for k in [1, 2, 3]:
            if np.array_equal(np.rot90(inp, k), out):
                # Verify on all examples
                valid = True
                for ex2 in train_examples[1:]:
                    inp2 = np.array(ex2['input'])
                    out2 = np.array(ex2['output'])
                    if not np.array_equal(np.rot90(inp2, k), out2):
                        valid = False
                        break
                
                if valid:
                    test_arr = np.array(test_input)
                    return np.rot90(test_arr, k).tolist()
        
        # Try horizontal and vertical flips
        for axis in [0, 1]:
            if np.array_equal(np.flip(inp, axis), out):
                valid = True
                for ex2 in train_examples[1:]:
                    inp2 = np.array(ex2['input'])
                    out2 = np.array(ex2['output'])
                    if not np.array_equal(np.flip(inp2, axis), out2):
                        valid = False
                        break
                
                if valid:
                    test_arr = np.array(test_input)
                    return np.flip(test_arr, axis).tolist()
        
        # Try transpose
        if np.array_equal(inp.T, out):
            valid = True
            for ex2 in train_examples[1:]:
                inp2 = np.array(ex2['input'])
                out2 = np.array(ex2['output'])
                if not np.array_equal(inp2.T, out2):
                    valid = False
                    break
            
            if valid:
                test_arr = np.array(test_input)
                return test_arr.T.tolist()
        
        return None
        
    except Exception as e:
        return None

# ============================================================
# STRATEGY 4: SCALING OPERATIONS
# ============================================================

def try_scaling(train_examples: List[Dict], test_input: List[List[int]]) -> Optional[List[List[int]]]:
    """
    Detect uniform upscaling or downscaling operations.
    """
    try:
        if len(train_examples) == 0:
            return None
        
        ex = train_examples[0]
        inp = np.array(ex['input'])
        out = np.array(ex['output'])
        
        # Calculate scale factor
        if inp.shape[0] == 0 or inp.shape[1] == 0:
            return None
        
        h_scale = out.shape[0] / inp.shape[0]
        w_scale = out.shape[1] / inp.shape[1]
        
        # Only handle uniform integer scaling
        if h_scale != w_scale or h_scale != int(h_scale):
            return None
        
        scale = int(h_scale)
        
        # Check if it's simple upscaling (each cell becomes scale x scale block)
        if scale > 1:
            # Upscaling
            expected = np.repeat(np.repeat(inp, scale, axis=0), scale, axis=1)
            if not np.array_equal(expected, out):
                return None
            
            # Verify on all examples
            for ex2 in train_examples[1:]:
                inp2 = np.array(ex2['input'])
                out2 = np.array(ex2['output'])
                expected2 = np.repeat(np.repeat(inp2, scale, axis=0), scale, axis=1)
                if not np.array_equal(expected2, out2):
                    return None
            
            # Apply to test
            test_arr = np.array(test_input)
            result = np.repeat(np.repeat(test_arr, scale, axis=0), scale, axis=1)
            return result.tolist()
        
        return None
        
    except Exception as e:
        return None

# ============================================================
# MULTI-STRATEGY SOLVER
# ============================================================

def multi_strategy_solve(task: Dict) -> List[List[List[int]]]:
    """
    Apply multiple solving strategies to a task.
    Returns up to 2 candidate solutions.
    
    Inspired by multi-approach mathematical reasoning:
    - Try different solution methods
    - Verify consistency with training examples
    - Select best candidates
    """
    train_examples = task['train']
    test_input = task['test'][0]['input']
    
    candidates = []
    
    # Strategy 1: Color Mapping
    result = try_color_mapping(train_examples, test_input)
    if result is not None:
        candidates.append(result)
    
    # Strategy 2: Tiling
    result = try_tiling(train_examples, test_input)
    if result is not None:
        candidates.append(result)
    
    # Strategy 3: Spatial Transformations
    result = try_spatial_transform(train_examples, test_input)
    if result is not None:
        candidates.append(result)
    
    # Strategy 4: Scaling
    result = try_scaling(train_examples, test_input)
    if result is not None:
        candidates.append(result)
    
    # Remove duplicate candidates
    unique_candidates = []
    seen = set()
    for candidate in candidates:
        candidate_str = str(candidate)
        if candidate_str not in seen:
            seen.add(candidate_str)
            unique_candidates.append(candidate)
    
    # Ensure we have exactly 2 attempts
    if len(unique_candidates) == 0:
        # Fallback: return input as-is
        unique_candidates.append(test_input)
        unique_candidates.append(test_input)
    elif len(unique_candidates) == 1:
        # Duplicate the single candidate
        unique_candidates.append(unique_candidates[0])
    
    return unique_candidates[:2]

# ============================================================
# CREATE SUBMISSION
# ============================================================

print("\nGenerating predictions for all tasks...")
submission = {}

for idx, (task_id, task) in enumerate(test_challenges.items(), 1):
    if idx % 50 == 0:
        print(f"  Processed {idx}/{len(test_challenges)} tasks...")
    
    try:
        # Solve the task
        attempts = multi_strategy_solve(task)
        
        # Format for each test output in this task
        task_outputs = []
        for test_item in task['test']:
            task_outputs.append({
                'attempt_1': attempts[0],
                'attempt_2': attempts[1]
            })
        
        submission[task_id] = task_outputs
        
    except Exception as e:
        print(f"  Error on {task_id}: {e}")
        # Fallback: submit input
        fallback = task['test'][0]['input']
        task_outputs = []
        for test_item in task['test']:
            task_outputs.append({
                'attempt_1': fallback,
                'attempt_2': fallback
            })
        submission[task_id] = task_outputs

# ============================================================
# SAVE SUBMISSION
# ============================================================

print("\nSaving submission...")
with open('submission.json', 'w') as f:
    json.dump(submission, f)

print(f"\n{'='*60}")
print(f"SUBMISSION COMPLETE")
print(f"{'='*60}")
print(f"Tasks processed: {len(submission)}")
print(f"File: submission.json")
print(f"\nReady to submit!")
# </parameter>
# </invoke>

Loading test challenges...
Loaded 240 test tasks

Generating predictions for all tasks...
  Processed 50/240 tasks...
  Processed 100/240 tasks...
  Processed 150/240 tasks...
  Processed 200/240 tasks...

Saving submission...

SUBMISSION COMPLETE
Tasks processed: 240
File: submission.json

Ready to submit!
